# Generation of APEX Weather Files
This notebook contains scripts to format climate data to .dly and .hly files for APEX from NASA POWER data. Follow the steps below to download NASA POWER data relevant to your project and load functions to generate new (or add to existing) APEX weather files. 
<br><br>
The following functions are included:
<br> - apex2dly: generate .dly file from NASA data (options to also generate .hly and .dly files and add spinup time)
<br> - nasa2hly: generate .hly file from NASA data
<br> - dly2hly: generate .hly file via uniform aggregated precip from existing .dly file
<br> - dly2wp1: generate .wp1 file from existing .dly file (requires wxpm v.3020)
<br> - add_spinup: duplicates first year of data 3 times to account for model spinup
<br><br>
<i>WARNING: MUST ENTER NEW STATIONS IN CORRESEPONDING .DAT INPUT FILES


#### 1. Load necessary modules and functions

In [ ]:
import os
import requests
import pandas as pd
from io import StringIO
import datetime
import shutil
import subprocess
import numpy as np

#### 2. Load NASA data using API

a. Set parameters for API search

In [3]:
# **EDIT THIS BLOCK** to match site of interest
longitude = '-80.0598'
latitude = '40.443'
start = '20130101'
end = '20221231'
elevation = '372.8'

b. Get daily data:

In [4]:
base_url = 'https://power.larc.nasa.gov'
endpoint = '/api/temporal/daily/point'

# Define the query parameters
params = {
    'parameters': ['T2M_MAX,T2M_MIN,RH2M,ALLSKY_SFC_SW_DWN,PRECTOTCORR,WS10M'],
    'community': 'AG',
    'longitude': longitude,
    'latitude': latitude,
    'start': start,
    'end': end,
    'format': 'CSV'
}

# Make the GET request to the API
response = requests.get(f"{base_url}{endpoint}", params=params)
# 
dly_NASA = response.text

c. Get hourly data (optional):

In [5]:
base_url = 'https://power.larc.nasa.gov'
endpoint = '/api/temporal/hourly/point'

# Define the query parameters
params = {
    'parameters': ['PRECTOTCORR'],
    'community': 'AG',
    'longitude': longitude,
    'latitude': latitude,
    'start': start,
    'end': end,
    'format': 'CSV',
}

# Make the GET request to the API
response = requests.get(f"{base_url}{endpoint}", params=params)

hly_NASA = response.text

#### 3. Load all functions

In [ ]:
def convert_julian(year,doy):
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=doy - 1)
    return date.month, date.day
# ----------------------------------------------------------------------
def nasa2dly(dly_fp, dly_fn, wxpm_fp=None, hly=None, spinup=None):
    '''
    formats nasa power data to APEX .dly (and others)
    
    parameters:
        dly_fp: directory path to where output weather files will be written
        dly_fn: filename for output
        wpxpm_fp (optional): fp to wxpm executable, wxpm-03082019, for monthly (wp1) generation
            --> note: file must contain both wxpm.exe and wxpmrun.DAT
        hly (optional): generate hly file
            --> 'uda' = generate from daily data using uniform disaggregation
            --> 'nasa' = generate directly from nasa hourly data
        spinup (optional): duplicates first year of data 3x to account for model spinup time
    ''' 

    # read data into df
    dly_df = pd.read_csv(StringIO(dly_NASA), delimiter=',', skiprows=14)

    # add columns MONTH and DAY
    dly_df['MONTH'], dly_df['DAY'] = zip(*dly_df.apply(lambda row: convert_julian(int(row['YEAR']), int(row['DOY'])), axis=1))
    
    # set column names to match .dly
    dly_colnames = {'T2M_MAX':'TMAX',
                    'T2M_MIN':'TMIN',
                    'PRECTOTCORR':'PRCP',
                    'RH2M':'RH',
                    'ALLSKY_SFC_SW_DWN':'SRAD',
                    'WS10M':'WSPD'}
    dly_df = dly_df.rename(columns=dly_colnames)
    
    # rearrange columns
    dly_df = dly_df[['YEAR','MONTH','DAY','SRAD','TMAX','TMIN','PRCP','RH','WSPD']]
    
    # build path to .dly file
    dly_path = os.path.join(dly_fp, dly_fn)

    # write .dly file 
    with open(dly_path, 'w') as file:
        for _, row in dly_df.iterrows():
            file.write(f'{int(row['YEAR']):6d} {int(row['MONTH']):3d} {int(row['DAY']):3d} {float(row['SRAD']):5.1f} {float(row['TMAX']):5.1f} {float(row['TMIN']):5.1f} {float(row['PRCP']):5.1f} {float(row['RH']):5.1f} {float(row['WSPD']):5.1f}\n')

    
    # generate wp1 file (if specified)
    if wxpm_fp is not None:
       dly2wp1(dly_fp, dly_fn, wxpm_fp)

    # generate .hly file via equal disaggregation (if specified)
    if hly == 'uda':
        dly2hly(dly_fp, dly_fn)
    else:
        hly_fn = dly_fn.replace('.dly', '.hly')
        nasa2hly(dly_fp, hly_fn)
    
    # add spinup time (if specified)
    if spinup is not None:
        add_spinup(dly_fp, dly_fn)
        if hly is not None:
            hly_fn = dly_fn.replace('.dly', '.hly')
            add_spinup(dly_fp, hly_fn)
# ----------------------------------------------------------------------
def nasa2hly(hly_fp,hly_fn):
    '''
    converts .hly file to .nasa format
    parameters:
        hly_fp: path to .hly file
        hly_fn: name of .hly file
    '''
    # read data into df
    hly_df = pd.read_csv(StringIO(hly_NASA), delimiter=',', skiprows=9)
    hly_colnames = {'YEAR':'YEAR',
                    'MO':'MONTH',
                    'DY':'DAY',
                    'HR':'HOUR',
                    'PRECTOTCORR':'RFDT'}
    hly_df = hly_df.rename(columns=hly_colnames)

    # convert to mm for APEX
    hly_df['RFDT'] = hly_df['RFDT']/24
    hly_df['RFDT'] = hly_df.groupby(['YEAR','MONTH','DAY'])['RFDT'].cumsum() # cumulative
    
    # adjust index for hourly
    hly_df['HOUR'] = hly_df['HOUR'] + 1
    
    # build path to .dly file
    hly_path = os.path.join(hly_fp, hly_fn)

    # write .dly file 
    with open(hly_path, 'w') as file:
            for _, row in hly_df.iterrows():
                file.write(f'{int(row['YEAR']):4d} {int(row['MONTH']):3d} {int(row['DAY']):3d} {int(row['HOUR']):9d} {float(row['RFDT']):9.3f}\n')

# ----------------------------------------------------------------------
def dly2hly(dly_fp, dly_fn):
    '''
    generates .hly file from .dly file using uniform disaggregation

    parameters:
        dly_fp: path to .dly file
        dly_fn: name of .dly file
    '''

    # set path to dly file
    dly_path = os.path.join(dly_fp, dly_fn)

    # read dly file into df
    colspecs = [(0, 6), (7, 10), (11, 14), (15, 20), (21, 26), (27, 32), (33, 38), (39, 44), (45, 50)]
    dly_df = pd.read_fwf(dly_path, colspecs=colspecs, header=None)
    dly_colnames = ['YEAR','MONTH','DAY','SRAD','TMAX','TMIN','PRCP','RH','WSPD']
    dly_df.columns = dly_colnames

    # create hly df where each row is duplicated 24 times
    hly_df = dly_df.loc[dly_df.index.repeat(24)].copy()

    # add column for hour of the day
    hly_df['HOUR'] = np.tile(np.arange(1,25), len(dly_df))

    # create new column for hourly precip
    hly_df['RFDT'] = hly_df['PRCP']/24
    hly_df['RFDT'] = hly_df.groupby(['YEAR','MONTH','DAY'])['RFDT'].cumsum() # cumulative

    # drop unncessary columns
    hly_df_filtered = hly_df.drop(columns=['SRAD','TMAX','TMIN','PRCP','RH','WSPD'])

    # build path to hly file
    hly_fn = os.path.splitext(dly_fn)[0] + '.hly'
    hly_path = os.path.join(dly_fp, hly_fn)

    # format and write into hly file
    with open(hly_path, 'w') as file:
            for _, row in hly_df_filtered.iterrows():
                file.write(f'{int(row['YEAR']):4d} {int(row['MONTH']):4d} {int(row['DAY']):4d} {int(row['HOUR']):10d} {float(row['RFDT']):10f}\n')

#----------------------------------------------------------------

def dly2wp1(dly_fp, dly_fn, wxpm_fp):
    
    '''
    converts .dly file to .wp1 using wxpm
    
    parameters:
        dly_fp: path to file containing source .dly (and destination for wp1)
        dly_fn: .dly filename
        wxpm_fp: path to file containing wxpmrun.dat and wxpm.exe
    '''
    
    # set file paths
    src_dly = os.path.join(dly_fp, dly_fn)
    wxpm_dly = os.path.join(wxpm_fp, dly_fn)
    wxpm_run = os.path.join(wxpm_fp, 'wxpmrun.dat')
    wxpm_exe = os.path.join(wxpm_fp, 'wxpm.exe')
    wp1_fn = os.path.splitext(dly_fn)[0] + '.wp1'
    out_wp1 = os.path.join(dly_fp, wp1_fn)

    # copy DLY file into WXPM folder
    shutil.copy(src_dly, wxpm_dly)

    # update wxpmrun.dat
    df = pd.read_csv(src_dly)
    yr1 = df.iloc[:,0].min() # get start year of data
    base_name = os.path.splitext(dly_fn)[0] # get name of dly file without extension
    with open(wxpm_run, 'r') as f:
        lines = f.readlines()
    lines[0] = f"{base_name} {yr1}\n"
    with open(wxpm_run, 'w') as f:
        f.writelines(lines)

    # run wxpm.exe
    subprocess.run([wxpm_exe], cwd=wxpm_fp)

    # copy generated wp1 to dly folder
    shutil.copy(os.path.join(wxpm_fp, wp1_fn), out_wp1)

# ----------------------------------------------------------------------
def add_spinup(fp,fn):
    
    '''
    adds spinup time to .dly or .hly file by repeating the first year of data three times

    warnings:
    - duplicated years should be excluded from output analysis
    - "year" column in duplicated years is modified based on their position in the dataframe
        --> these values do not represent actual years - they are just labeled to ensure proper model execution
    
    parameters:
        fp: path to weather file
        fn: name of weather file
    '''

    file_path = os.path.join(fp, fn)

    # read data in as df
    # weather_df = pd.read_csv(file_path, header=None, sep='\s+')
    if file_path.split('.')[1] == 'dly':
        colspecs = [(0, 6), (7, 10), (11, 14), (15, 20), (21, 26), (27, 32), (33, 38), (39, 44), (45, 50)]
        weather_df = pd.read_fwf(file_path, colspecs=colspecs, header=None)
        weather_df.columns =['YEAR','MONTH','DAY','SRAD','TMAX','TMIN','PRCP','RH','WSPD']
        file_type = 'dly'
    if file_path.split('.')[1] == 'hly':
        colspecs = [(0, 4), (5, 8), (9, 12), (13, 22), (23, 32)]
        weather_df = pd.read_fwf(file_path, colspecs=colspecs, header=None)
        weather_df.columns =['YEAR','MONTH','DAY','HOUR','RFDT']
        file_type = 'hly'

    # get first year of data (column 1)
    yr1 = weather_df.iloc[:,0].min()

    # duplicate first year 3x, adjusting number of year
    yr1_data = weather_df[weather_df.iloc[:,0] == yr1]
    duplicated_data = pd.concat([yr1_data]*3, ignore_index=True)

    # adjust year column for duplicated data
    for i in range(3):
        duplicated_data.iloc[i*len(yr1_data):(i+1)*len(yr1_data), 0] = yr1 - (3-i)


    # add duplicated data to existing file
    weather_df = pd.concat([duplicated_data, weather_df], ignore_index=True)

    # write updated file
    # new file name keeps the same name (and extension) but adds '_spinup'
    spinup_fn = fn.split('.')[0] + '_spinup.' + fn.split('.')[1]
    spinup_path = os.path.join(fp, spinup_fn)

    # write file according to file extension
    with open(spinup_path, 'w') as file:
        for _, row in weather_df.iterrows():
            if file_type == 'dly':
                file.write(f'{int(row['YEAR']):6d} {int(row['MONTH']):3d} {int(row['DAY']):3d} {float(row['SRAD']):5.1f} {float(row['TMAX']):5.1f} {float(row['TMIN']):5.1f} {float(row['PRCP']):5.1f} {float(row['RH']):5.1f} {float(row['WSPD']):5.1f}\n')
            if file_type == 'hly':
                file.write(f'{int(row['YEAR']):4d} {int(row['MONTH']):3d} {int(row['DAY']):3d} {int(row['HOUR']):9d} {float(row['RFDT']):9.3f}\n')
                

#### 4. Generate files

##### APEX2DLY: Daily & More
This function generates .dly files from NASA data downloaded using the API method above. You can also enter some optional arguments to generate accompanying monthly (.wp1) and hourly (.hly) files, either directly from NASA data or uniformly aggregated from the daily data, as well as add three years years of spinup time. <br><br>Note: The filepath for your .dly file should be where the rest of your APEX input and executable files are stored. <br><br>The optional functionalities can also be used independently (see blocks below for individual functions).

In [ ]:
dly_fp = 'C:\\APEX\\apex1501-20241028\\test' # replace with folder path where file will be written
dly_fn = 'test.dly' # replace with desired file name

# optional (uncomment to use)
#wxpm_fp = 'C:\\APEX\\wxpm-03082019\\' # edit to match path to wxpm folder for .wp1 generation
#hly = 'uda' # generate hly file using uniform disaggregation ('uda') or directly from nasa data ('nasa')
#spinup = 'y' # duplicate first year of data 3x to account for model spinup time

nasa2dly(dly_fp, dly_fn) # adjust/add arguments as needed

##### NASA2HLY: Hourly (from NASA)

This function generates .hly files from NASA data downloaded using the API method above. See dly2hly for hourly data uniformly aggregated from daily NASA data.


In [ ]:
hly_fp = 'C:\\APEX\\apex1501-20241028\\test' # replace with folder path where file will be written
hly_fn = 'test.hly' # replace with desired file name

nasa2hly(hly_fp,hly_fn)

##### DLY2HLY: Hourly (Uniformly Aggregated)
This function generates .hly files, uniformly aggregated from an existing .dly file.
<br> *note: hourly file with share the same name as the source .dly file (but with .hly extension)

In [ ]:
dly_fp = 'C:\\APEX\\apex1501-20241028\\test' # replace with folder path where source .dly exists
dly_fn = 'test.dly' # replace with source .dly filename

dly2hly(dly_fp, dly_fn)

##### DLY2WP1: Monthly

This function generates .wp1 files from an existing .dly file, using APEX's monthly weather file generation program, WXPM v.3020 (can be downloaded at https://epicapex.tamu.edu/software/). 
<br> *note: monthly file with share the same name as the source .dly file (but with .wp1 extension)

In [ ]:
dly_fp = 'C:\\APEX\\apex1501-20241028\\test' # replace with folder path where source .dly exists
dly_fn = 'test.dly' # replace with source .dly filename
wxpm_fp = 'C:\\APEX\\wxpm-03082019\\' # replace with path to wxpm folder

dly2wp1(dly_fp, dly_fn, wxpm_fp)

##### add_spinup: Spinup time

This function adds spinup time to an existing .dly or .hly file by repeating the first year of data three times.
<br> *warnings: <br> - duplicated years should be excluded from output analysis (first three years) <br> - "year" column in duplicated years is modified based on their position in the dataframe; these values do not represent actual years - they are just labeled to ensure proper model execution



In [ ]:
fp = 'C:\\APEX\\apex1501-20241028\\test' # replace with folder path where source .dly or .hly exists
fn = 'test.dly' # replace with source .dly or .hly filename

add_spinup(fp,fn)